# AutoEncoder


## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_recall_curve

from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from keras.optimizers import Adam

### (2) 필요 함수들 생성

#### ① reconstruction error plot

* input과 output의 차이(mse)를 계산하고
* 시각화 합니다.



In [ ]:
def recon_err_plot(x, x_pred, y, threshold=0):
    # treshold : 우리가 지정해줘야 함.(어떻게?)

    # 재구성 오차 계산
    mse = np.mean(np.power(x - x_pred, 2), axis=1)
    error_df = pd.DataFrame({'recon_err': mse, 'y': y})
    error_df = error_df.reset_index()

    # 재구성 오차 그래프
    groups = error_df.groupby('y')
    fig, ax = plt.subplots()
    for name, group in groups:
        ax.plot(group.index, group.recon_err, marker='o', ms=3.5, linestyle='',
                label= "Abnormal" if name == 1 else "Normal")
    ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
    ax.legend()
    plt.title("Reconstruction error for each data point")
    plt.ylabel("Reconstruction error")
    plt.xlabel("Data point index")
    plt.show()

    return error_df

#### ② precision, recall, f1 curve

* sklearn에서는 precision, recall curve만 제공됩니다.
* 그래서, f1 curve도 추가해서 구하고, plot을 그립니다.



In [ ]:
def prec_rec_f1_curve(y, score, pos = 1) :

    # thresholds를 조정해가면서, precision, recall을 뽑고, f1 score 계산
    precision, recall, thresholds  = precision_recall_curve(y, score, pos_label = pos)
    f1 = 2 / (1/precision + 1/recall)

    # thresholds에 따라 조금씩 계산된 precision, recall, f1 score 그래프로 그리기
    plt.figure(figsize = (8, 6))
    plt.plot(thresholds, np.delete(precision, -1), label = 'precision')
    plt.plot(thresholds, np.delete(recall, -1), label = 'recall')
    plt.plot(thresholds, np.delete(f1, -1), label = 'f1')
    plt.xlabel('Anomaly Score')
    plt.legend()
    plt.grid()
    plt.show()

    return precision, recall, f1, thresholds

#### ③ threshold로 잘랐을 때, 분류 평가 함수


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def classification_report2(y, pred, threshold):

    # 재구성 오차(pred)를 threshold로 잘라서 1, 0 만듬
    pred_temp = np.where(pred >= threshold , 1, 0)

    print('< confusion matrix >\n')
    print(confusion_matrix(y, pred_temp))
    print('\n' + '='*60 + '\n')

    print('< classification_report >\n')
    print(classification_report(y, pred_temp))

#### ④ DL 학습곡선 그리기


In [ ]:
def plot_learning_curve(history) :
    plt.figure(figsize = (10,6))
    plt.plot(history['loss'], label='Train', marker = '.')
    plt.plot(history['val_loss'], label='Validation', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.grid()
    plt.show()

### (3) 데이터셋 불러오기

In [ ]:
# secom
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/secom_9.csv"
data = pd.read_csv(path)

data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data['label'].value_counts()

In [ ]:
data['label'].value_counts(normalize = True)

## 2.데이터 준비

### (1) 데이터 분할

In [ ]:
target = 'label'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state=20)

In [ ]:
x_train.shape, y_train.shape

### (2) 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

### (3) Normal 데이터만 뽑기

In [ ]:
# 학습을 위해서는 Normal 데이터만 이용
x_train0 = x_train[y_train == 0]

In [ ]:
x_train0.shape

## 3.모델링 : AE

### (1) 학습

#### 1) 모델 설계

In [ ]:
input_dim = x_train0.shape[1]
input_dim

In [ ]:
input_dim = x_train0.shape[1] #num of columns

clear_session()
autoencoder = Sequential([Dense(32, input_shape = (input_dim,), activation = 'relu'),
                          Dense(16, activation="relu"),
                          Dense(32, activation='relu'),
                          Dense(input_dim)])

autoencoder.summary()

#### 2) compile + 학습

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')

history = autoencoder.fit(x=x_train0, y=x_train0,
                          epochs=50, batch_size=64,
                          validation_split = .2).history

In [ ]:
plot_learning_curve(history)

### (2) 예측 및 Reconstruction Error 계산

#### 1) 예측

In [ ]:
pred = autoencoder.predict(x_val)
pred

In [ ]:
x_val.shape, pred.shape

#### 2) 재구성 오차 계산

In [ ]:
result = recon_err_plot(x_val, pred, y_val, .01)

In [ ]:
result.head()

#### 3) Threshold 찾기 : precision_recall_f1_curve

In [ ]:
precision, recall, f1, thresholds = prec_rec_f1_curve(result['y'], result['recon_err'])

In [ ]:
thres_f1_max = thresholds[np.argmax(f1)]
thres_f1_max

#### 4) f1을 최대화 하는 threshold 값으로 잘라서 평가

In [ ]:
classification_report2(result['y'], result['recon_err'], thres_f1_max)

## 4.🤔실습 : 사기거래 탐지

### (1) 데이터 준비

* 데이터 로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/creditcard_simple.csv"
data = pd.read_csv(path)

target = 'fraud'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

In [ ]:
data.head()

* 데이터 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 6000, random_state=20)

* 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [ ]:
x_train.shape

* Normal 데이터만 저장

### (2) 모델링 : AE

#### 1) 학습

* 모델 설계
    * 아래 구조를 참조하되 여러분이 원하는 대로 생성을 해 봅시다.

_________________________________________________________________
 Layer (type)                Output Shape              
_________________________________________________________________
dense (Dense)               (None, 32)                     
                                                                 
 dense_1 (Dense)             (None, 16)                     
                                                                 
 dense_2 (Dense)             (None, 8)                      
                                                                 
 dense_3 (Dense)             (None, 16)                     
                                                                 
 dense_4 (Dense)             (None, 32)                     
                                                                 
 dense_5 (Dense)             (None, 19)                     
_________________________________________________________________


In [ ]:
x_train0.shape

* compile + 학습

#### 2) 예측 및 검증

* 예측

* 재구성 오차 그래프

* precision_recall_f1_curve

* f1을 최대화 하는 threshold 값으로 평가

👏수고하셨습니다~!👏